In [80]:
import pandas as pd
import numpy as np
import networkx as nx
import community as community_louvain

In [81]:
edges_df = pd.read_csv('../datasets/wits/el_no_missing.csv')
locations_df = pd.read_csv('../datasets/wits/locations.csv')
nodes_df = pd.read_csv('../datasets/wits/nl_no_missing.csv')

In [82]:
edges_df.head(3)

,source,target,weight
0,NPL,LBR,97.061
1,NPL,AZE,46.502
2,NPL,JOR,25.551


In [83]:
nodes_df.head(3)

,country_iso3,gdp_us_dollar,inflation_rate,gdp_growth,colonizer,area,landlocked,continent,langoff_1,population,gdp_per_capita,life_expectancy,gni_atlas,agriculture_forestry_fishing_of_gdp,industry_of_gdp,merchandise_of_gdp,net_barter_of_trade,foreign_direct_investment_inflows,happiness
0,AFG,1.780429e+10,11.804186,0.426355,USA,652225,1,Asia,Persian,30117411,591.162798,61.553,1.607712e+10,23.743664,22.740252,38.704149,144.751869,5.217342e+07,0.721
1,AGO,1.117897e+11,13.482468,3.471976,PRT,1246700,0,Africa,Portuguese,24220660,4615.468219,56.330,8.260603e+10,5.845681,56.026515,78.305971,244.329152,-3.023771e+09,0.708
2,ALB,1.289077e+10,3.429123,2.545322,NONE,28748,0,Europe,Albanian,2905195,4437.142885,76.914,1.280372e+10,18.226765,24.484408,56.862411,94.337202,1.048707e+09,0.568


In [84]:
locations_df.head(3)

,Country,Alpha-2code,Alpha-3code,Numericcode,Latitude(average),Longitude(average)
0,Afghanistan,AF,AFG,4,33.0,65.0
1,Albania,AL,ALB,8,41.0,20.0
2,Algeria,DZ,DZA,12,28.0,3.0


In [86]:
nodes_df = pd.merge(left=nodes_df, right=locations_df, how='inner', left_on='country_iso3', right_on='Alpha-3code')

In [93]:
countries = nodes_df['country_iso3'].unique()

In [95]:
trade_data = np.zeros((len(countries), len(countries)))
for i in range(len(countries)):
    for j in range(len(countries)):
        temp = edges_df[
            (edges_df['source'] == countries[i]) & (edges_df['target'] == countries[j])
        ]
        if temp.shape[0] != 0:
            trade_data[i, j] += temp['weight']
            trade_data[j, i] += temp['weight']

In [101]:
g = nx.Graph()

In [102]:
for i in range(len(countries)):
    for j in range(i + 1, len(countries)):
        if trade_data[i, j] != 0:
            g.add_edge(countries[i], countries[j], weight = trade_data[i, j])

In [103]:
cols = list(nodes_df.columns)
cols.remove('country_iso3')

In [104]:
partition = community_louvain.best_partition(g)

In [105]:
nodal_attributes = dict()
for index, row in nodes_df.iterrows():
    nodal_attributes[row['country_iso3']] = dict()
    for col in cols:
        nodal_attributes[row['country_iso3']][col] = row[col]
    nodal_attributes[row['country_iso3']]['community'] = partition[row['country_iso3']]

In [106]:
nx.set_node_attributes(g, nodal_attributes)

In [107]:
nx.write_gexf(G=g, path='../datasets/wits/trade_network.gexf')